# Mount drive and bucket
Todo: Remove in public version

In [1]:
# Check if the notebook is run in Google Colab
import sys

COLAB = 'google.colab' in sys.modules

In [2]:
# Uncomment this cell to provide logging output to a file in addition to the lab in the browser
# This also keeps logging if the browser window is closed

# import datetime
# import logging

# nblog = open(f"/content/CryptoCrystalBall/Data/logs/nb_{datetime.datetime.utcnow().strftime('%Y%m%d_%H%M')}.log", "a+")
# print(nblog)
# sys.stdout.echo = nblog
# sys.stderr.echo = nblog

# %autosave 60

In [3]:
if COLAB:
  from google.colab import auth
  auth.authenticate_user()
else:
    print("Run the command!")
  #Todo #bring the command inside the notebook
  #run this terminal inside docker: gcloud auth login b.girsule@gmail.com --no-launch-browser

Run the command!


In [4]:
# Check if the tf version is 2.10.0, this is required to use the 'ignore_class' in the  SparseCategoricalCrossentropy
import tensorflow as tf

if '2.10.0' != tf.__version__:
  !pip uninstall tensorflow -y
  !pip install tensorflow-gpu==2.10.0
  please_restart_the_runtime
else:
  print("TF Version is 2.10.0, ok!")

2023-04-15 05:45:12.514711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 05:45:12.741525: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-15 05:45:12.904944: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-15 05:45:13.757158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

TF Version is 2.10.0, ok!


In [5]:
from tensorflow import keras
from tensorflow.python.lib.io import file_io

from keras.layers import Input, Dense, Flatten, UnitNormalization, Permute, Conv1D, UpSampling1D, Add, LSTM, Concatenate, Multiply, Activation, Attention, Dropout
from keras.models import Model
from keras.optimizers import Adam

In [6]:
import os
from os import listdir
from os.path import isfile, join

In [7]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import gc
import random
import datetime

In [8]:
# Set the google cloud bucket data
project_id = 'tweetprediction'
bucket_name = 'crypto_nlp_training'

In [9]:
# Set the checkpoint path for saving train progress
if COLAB:
    raise NotImplementedError("Google Buckets have to be implemented in this notebook before using them in Colab!")
    CHECKPOINT_PATH = f"gs://{bucket_name}/chk/"
else:
    CHECKPOINT_PATH = f"/content/bigdata/chk/"

In [10]:
# Check if the notebook is run in google colab, if so, clone the repo
if COLAB:
    print("Running in Colab")

    # Clone the whole repo to get all data and code if not already done
    if not os.path.exists("/content/CryptoCrystalBall"):
      !git clone https://github.com/girsigit/CryptoCrystalBall

    %cd /content

In [11]:
# Try importing the Ta-Lib library, if this fails, try to install it and
# import it again afterwards
try:
    import talib
except:
    !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzvf ta-lib-0.4.0-src.tar.gz
    %cd ta-lib
    !./configure --prefix=/usr
    !make
    !make install
    !pip install Ta-Lib
    %cd ..

    import talib

In [12]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

---
# Add custom import path for DataStreamCreator and IndicatorCalculator

These libs are not in the standard python directory, so their paths have to be added to the import paths

In [13]:
# Get the current directory
current_dir = os.getcwd()
current_dir_splitted = ["", "content"]
current_dir_splitted

['', 'content']

In [14]:
# Create the import directories for the DataStreamCreator and the IndicatorCalculator
dsc_dir = '/content/CryptoCrystalBall/DataStreamCreator'
print(f"dsc_dir: {dsc_dir}")

ind_dir = '/content/CryptoCrystalBall/IndicatorCalculator'
print(f"ind_dir: {ind_dir}")

# Add them to the import paths
sys.path.insert(0, dsc_dir)
sys.path.insert(0, ind_dir)

dsc_dir: /content/CryptoCrystalBall/DataStreamCreator
ind_dir: /content/CryptoCrystalBall/IndicatorCalculator


In [15]:
# Import the actual classes
from IndicatorCalculator import IndicatorCalculator
import DataStreamCreator

---
# Define all the parameters and variables

In [16]:
# # Define the tick data path
# DATA_PATH = os.path.join(os.sep, *current_dir_splitted, 'DemoData')
# print(f"DATA_PATH: {DATA_PATH}")

In [17]:
# # Define the chart image save path
# IMG_SAVE_PATH = os.path.join(os.sep, *current_dir_splitted, 'Documentation', 'Images')
# print(f"IMG_SAVE_PATH: {IMG_SAVE_PATH}")

In [18]:
# Define a global random seed
RANDOM_SEED = 42

# Set the seed in np
np.random.seed(RANDOM_SEED)

In [19]:
# X_BLOCK_LENGHT defines how far into the past a 'slice of a chart' shall be
# See: https://github.com/girsigit/CryptoCrystalBall/tree/main/DataStreamCreator#xblockgenerator

# For this ML architecture, the X_BLOCK_LENGHT has to be a member of 2**n, otherwise the feature pyramid cannot be built correctly 

# X_BLOCK_LENGHT = 256
X_BLOCK_LENGHT = 128

In [20]:
# How many examples shall be processed at the same time, limited by GPU memory
BATCH_SIZE = 384 # for X_BLOCK_LENGHT = 256
BATCH_SIZE = 512 # for X_BLOCK_LENGHT = 128

In [21]:
# A fixed number of features is used
FEATURES = 160

In [22]:
# Finanical indicator timespans
# See: https://github.com/girsigit/CryptoCrystalBall/tree/main/IndicatorCalculator
SHORTSPAN = 7
MIDSPAN = 38
LONGSPAN = 50

In [23]:
# Additional settings for the data stream
# For this notebook, the calculation of pattern indicators is turned off
DATA_STREAM_PARAMETERS = {
    "calcPatternIndicators": False, # No patterns are used
    "calcVolumeInidators": False, # No volume indicators, these are wide spread and may disturb the classifer
    "dropna": True # Drop all tick/indicator table rows containing nan values instead of just replacing them by 0 (which would lead to wrong predictions)
}

# Load the dataset

In [24]:
# Define the dataset path
DATASET_PATH = "/content/dataset/ETF/tickdata/ticks/train"

# Get all file names
TRAIN_FILES = [os.path.join(DATASET_PATH,f) for f in listdir(DATASET_PATH) if isfile(join(DATASET_PATH, f)) and ".csv" in f ]

# Sort them (as a stable basis for randomizing afterwards)
TRAIN_FILES = sorted(TRAIN_FILES)

print(f"The train dataset contains {len(TRAIN_FILES)} files.")
print(TRAIN_FILES[:3])

The train dataset contains 911 files.
['/content/dataset/ETF/tickdata/ticks/train/DE0002635265.csv', '/content/dataset/ETF/tickdata/ticks/train/DE0002635273.csv', '/content/dataset/ETF/tickdata/ticks/train/DE0002635281.csv']


### Not required, dataset is already prepared
---
### Split the dataset into 3 parts
- The **train** set is used to train the machine-learning network then used to predicting future information
- The **test** set is used in the next step. On it a trading (entry/exit) strategy will be developed
- The **eval** set will then be used as an never-seen set to try and evalutate the combination of the ML network and the trading strategy

In [25]:
# # It uses a fixed random seed to allow this cell to be copied to other notebooks producing the same result
# SPLIT_RANDOM_SEED = 666

# TRAIN_SET_SIZE = 0.5
# TEST_SET_SIZE = 0.25
# # EVAL_SET_SIZE: The rest

# # Files are shuffled before splitting
# random.Random(SPLIT_RANDOM_SEED).shuffle(ALL_FILES)

# train_split_index = int(len(ALL_FILES) * TRAIN_SET_SIZE)
# print(f"train_split_index: {train_split_index}")

# test_split_index = int(len(ALL_FILES) * (TRAIN_SET_SIZE + TEST_SET_SIZE))
# print(f"test_split_index: {test_split_index}")

# TRAIN_FILES = ALL_FILES[:train_split_index]
# TEST_FILES = ALL_FILES[train_split_index:test_split_index]
# EVAL_FILES = ALL_FILES[test_split_index:]

# print(f"The train set contains {len(TRAIN_FILES)} files")
# print(f"The test set contains {len(TEST_FILES)} files")
# print(f"The eval set contains {len(EVAL_FILES)} files")

---
# Prepare data source

For training a neural network, first the data source has to be prepared. For this purpose, the method `FileListToDataStream` from the `DataStreamCreator` class is used. This method creates a stream of `X-Block` and `y-data` arrays out of a list of .csv file names, pointing to tick tables (called `EXAMPLE_FILE_PATHS` in this example). For details about `X-Blocks` and `y-data`, please refer to the documentation of the `XBlockGenerator` and the `YDataGenerator` under https://github.com/girsigit/CryptoCrystalBall/tree/main/DataStreamCreator.

<br>

As y values for training the ML network the possible future gain and it derivation will be used. The y data type `PARAM_DICT_TEMPLATE_Y_DATA_TYPE_PAST_FUTURE_GAIN` will provide both past and future gain, but the model will only be trained on the future gain, as the past gain can be easily calculated directly, also in a live application.

In [26]:
Y_TYPE_DICT = copy.deepcopy(DataStreamCreator.YDataGenerator.PARAM_DICT_TEMPLATE_Y_DATA_TYPE_PAST_FUTURE_GAIN)

Y_TYPE_DICT["gain_timespan"] = 60
# Y_TYPE_DICT["direction_ma_timespan"] = 14
# Y_TYPE_DICT["derivation_ma_timespan"] = 14

Y_TYPE_DICT

{'dataType': 3,
 'gain_timespan': 60,
 'direction_ma_timespan': 48,
 'derivation_ma_timespan': 48}

In [27]:
XShape = (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES)
XShape

(512, 128, 160)

In [28]:
# A python generator function has to be applied on the dataStream

def pythonGeneratorSignalsTraining():
  # Initialize the FileListToDataStream generator
  dataStreamTraining = DataStreamCreator.FileListToDataStream(fileList = TRAIN_FILES,
                                                      batch_size = BATCH_SIZE,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=True,
                                                      parallel_generators = np.min([BATCH_SIZE, 8]),
                                                      random_seed = RANDOM_SEED,
                                                      **DATA_STREAM_PARAMETERS
                                                      )

  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStreamTraining)
      _X = ne['X']
      _y = ne['y'][:,[1,3]]
    
      # Limit the y signal to avoid excess
      _y /= 10.0
      _y = np.clip(_y, -10.0, 10.0)

      yield (_X, _y)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [29]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
tfGenTraining = tf.data.Dataset.from_generator(pythonGeneratorSignalsTraining, 
                                               output_types = (tf.float32, tf.float32),
                                               output_shapes=(
                                                   (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES),
                                                   (BATCH_SIZE, 2)
                                                   )
                                               )

tfGenTraining = tfGenTraining.prefetch(buffer_size=32)
tfGenTraining

2023-04-15 05:45:17.193523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-15 05:45:17.269408: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-15 05:45:17.269584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-15 05:45:17.271804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

<PrefetchDataset element_spec=(TensorSpec(shape=(512, 128, 160), dtype=tf.float32, name=None), TensorSpec(shape=(512, 2), dtype=tf.float32, name=None))>

---
# Create the neural network

This approach uses a FPN (Feature-Pyramid-Network, https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c) architecture, followed by LSTM and Dense regressor heads to extract the future gain information.
The pyramid elements are built in a Res-Net style to provide a good information flow.


Additionally, it is equipped with Attention layers to focus on relevant information. Futhermore, it uses antialiasing to smooth out disturbance.


```
Todo: Create Image of network structure
```

In [30]:
# Create the CreateModelFPNWithAttention model
def CreateModelFPNWithAttention():
  # Define model parameters
  mp = {
      "NAME": "FPNWithAttentionBiggerTimebased",
      "VERSION": "3_FutureOnly_60days",
      "CNN_INITIAL_FILTERS": 64,
      "CNN_EXTRACTOR_LAYERS": 6,
      "CNN_INITIAL_KERNEL_SIZE": 2,
      "CNN_MAX_KERNEL_SIZE": 8,
      "FEATURE_PYRAMID_START_INDEX": 2,
      "HEAD_LSTM_SIZE": 64,
      "HEAD_DENSE_SIZE": 32
      }
  
  # Create the model input  
  inputTicksAndIndicators = Input(shape=(X_BLOCK_LENGHT, FEATURES), name='inputTicksAndIndicators', dtype='float32')

  # Apply dropout on the input to make the model robust
  inputTicksAndIndicators = Dropout(0.1, name='inputDropout')(inputTicksAndIndicators)

  # Normalize the input data, as it has a wide value spread (range of 1e8)
  normalized = UnitNormalization(name="UnitNormalizationInput", axis=-2)(inputTicksAndIndicators)

  # ----------------------------------------------------------------------------

  # A 1D-CNN pyramid is used to extract higher-level features out of the time dimension
  conv_layers = []
  aligned_layers = []
  feature_map_layers = []
  anti_aliased_layers = []
  anti_aliased_pyramid_indices = []
  regressor_layers = []

  finalFilters = mp["CNN_INITIAL_FILTERS"] * 2**(mp["FEATURE_PYRAMID_START_INDEX"])
  print(finalFilters) 

  for i in range(mp["CNN_EXTRACTOR_LAYERS"]):
    if 0 == i:
      source_layer = normalized
    else:
      source_layer = feature_conv
    
    filters = int(mp["CNN_INITIAL_FILTERS"]*(2**i))
    kernel_size = mp["CNN_INITIAL_KERNEL_SIZE"]*(2**i)
    kernel_size = int(np.min([mp["CNN_MAX_KERNEL_SIZE"], kernel_size]))
    strides = int(np.min([2,kernel_size]))

    print(f"Step {i}, using {filters} filters with a kernel size of {kernel_size} at strides of {strides}")

    # Create the feature extraction convolution
    res_net_element = Conv1D(filters=source_layer.shape[-1],
                          kernel_size=kernel_size,
                          strides=1, 
                          padding="same",
                          name=f"ResNet_{i}_Conv1D_1")(source_layer)
    
    res_net_element = Activation("relu",
                              name=f"ResNet_{i}_ReLU")(res_net_element)
    
    res_net_element = Conv1D(filters=source_layer.shape[-1],
                          kernel_size=kernel_size,
                          strides=1, 
                          padding="same",
                          name=f"ResNet_{i}_Conv1D_2")(res_net_element)
    
    res_net_element = Add(name=f"ResNet_{i}_Add")([res_net_element, source_layer])
        
    feature_conv = Conv1D(filters=filters,
                          kernel_size=kernel_size,
                          strides=strides, 
                          padding="same",
                          name=f"ResNet_{i}_Feature_Conv1D_FilterDimReduction")(res_net_element)

    attention_map = Attention(name=f"Attention_Layer_{i}")([feature_conv,feature_conv])
    print(f"attention_map.shape: {attention_map.shape}")

    feature_conv_attentioned = Multiply(name=f"Multiply_Apply_Attention_{i}")([feature_conv, attention_map])
    print(f"feature_conv_attentioned.shape: {feature_conv_attentioned.shape}")
    
    conv_layers.append(feature_conv_attentioned)

    # Apply the feature extraction pyramid
    if mp["FEATURE_PYRAMID_START_INDEX"] <= i:
      # Apply a 1x1 convolution to align the channel depth    
      if feature_conv_attentioned.shape[-1] != finalFilters:
        aligned_layer = Conv1D(filters=finalFilters,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              name=f"Channel_Depth_Alignment_{i}")(feature_conv_attentioned)
      else:
        aligned_layer = feature_conv_attentioned
      
      aligned_layers.append(aligned_layer)

  # Go top-down through the aligned_layers to create the feature_map_layers
  for i in range(mp["CNN_EXTRACTOR_LAYERS"]-1, mp["FEATURE_PYRAMID_START_INDEX"]-1, -1):
    aligned_layers_index = i - mp["FEATURE_PYRAMID_START_INDEX"]
    aligned_layer = aligned_layers[aligned_layers_index]

    print(f"Building feature_map_layers, step {i}. Shape of aligned_layer: {aligned_layer.shape}")

    # The highest-filtered layer is taken as feature map directly
    if i == mp["CNN_EXTRACTOR_LAYERS"]-1:
      feature_map_layers.append(aligned_layer)
      anti_aliased_layers.append(feature_map_layers[-1])
    else:
      # Take the last feature_map_layer and scale it by two
      upsampled = UpSampling1D(size=2,
                               name=f"Pyramid_Upsampling_{i}")(feature_map_layers[-1])

      # Add the aligned_layer
      added = Add(name=f"Pyramid_Add_{i}")([upsampled, aligned_layer])

      # Append as new feature_map_layer
      feature_map_layers.append(added)

      # Apply a convolution with a kernel size of 3 to "reduce the aliasing effect"
      anti_aliased_layer = Conv1D(filters=feature_map_layers[-1].shape[-1],
                                  kernel_size=3,
                                  strides=1,
                                  padding="same",
                                  name=f"Anti_Alias_{i}")(feature_map_layers[-1])

      anti_aliased_layers.append(anti_aliased_layer)

    # Helper storage to keep pyramid index consistent, also in predictor/regressor
    anti_aliased_pyramid_indices.append(i)

  # Apply the predictor head to each feature dimension layer
  for n in range(len(anti_aliased_layers)):
    predictor_input = anti_aliased_layers[n]

    # Get the pyramid index
    pyramid_index = anti_aliased_pyramid_indices[n]

    # Permute the Conv output back
    predictor_input = Permute((2, 1), name=f"PermutePredictorInput_{pyramid_index}")(predictor_input)

    predictor = LSTM(units=mp["HEAD_LSTM_SIZE"],
                     name=f"Predictor_Head_{pyramid_index}_LSTM")(predictor_input)
    
    # Direction and derivation regressor
    regressor = Dense(units=mp["HEAD_DENSE_SIZE"],
                      name=f"Predictor_Head_{pyramid_index}_Dense_Regressor",
                      activation='relu')(predictor)
    
    regressor_layers.append(regressor)
  
  # Add all regressors together
  regressor_conced = Concatenate(name="Concatenate_regressors")(regressor_layers)
  #regressor_conced = Dense(regressor_conced.shape[1], name="Combined_Regressor_1")(regressor_conced)
  #regressor_conced = Dense(regressor_conced.shape[1], name="Combined_Regressor_2")(regressor_conced)

  # The output are two values (direction and derivation) in the range [-1.0, 1.0]
  output = Dense(2, activation="tanh", name="Output")(regressor_conced)
  outputs = [output]

  # Combine it all into a model object
  model = Model(inputs=inputTicksAndIndicators, outputs=outputs, name=mp["NAME"] + "_" + str(mp["VERSION"]))

  return model, mp

model, model_config = CreateModelFPNWithAttention()
print(model_config)
model.summary(line_length=220)

256
Step 0, using 64 filters with a kernel size of 2 at strides of 2
attention_map.shape: (None, 64, 64)
feature_conv_attentioned.shape: (None, 64, 64)
Step 1, using 128 filters with a kernel size of 4 at strides of 2
attention_map.shape: (None, 32, 128)
feature_conv_attentioned.shape: (None, 32, 128)
Step 2, using 256 filters with a kernel size of 8 at strides of 2
attention_map.shape: (None, 16, 256)
feature_conv_attentioned.shape: (None, 16, 256)
Step 3, using 512 filters with a kernel size of 8 at strides of 2
attention_map.shape: (None, 8, 512)
feature_conv_attentioned.shape: (None, 8, 512)
Step 4, using 1024 filters with a kernel size of 8 at strides of 2
attention_map.shape: (None, 4, 1024)
feature_conv_attentioned.shape: (None, 4, 1024)
Step 5, using 2048 filters with a kernel size of 8 at strides of 2
attention_map.shape: (None, 2, 2048)
feature_conv_attentioned.shape: (None, 2, 2048)
Building feature_map_layers, step 5. Shape of aligned_layer: (None, 2, 256)
Building feature_

# Train the model

In [31]:
# Define the checkpoint name for storing training progress
CHKPNT_NAME = f"{model.name}_GPU_{X_BLOCK_LENGHT}LB"
CHKPNT_NAME

'FPNWithAttentionBiggerTimebased_3_FutureOnly_60days_GPU_128LB'

In [32]:
# Set an optimizer
optimizer = Adam(
    learning_rate=1e-03,
    epsilon=1e-06,
    beta_1=0.9,
    beta_2=0.98,
    decay=0.01)

In [33]:
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = ["mse"], 
    metrics=["mae"])

In [34]:
#@title CustomCallback
# This method stores the model weights every `save_freq` batches, or at least at the epoch's end

class CustomCallback(tf.keras.callbacks.Callback):
  def __init__(self, save_freq, val_freq, checkpoint_path, model_name, epoch_add=0):
    self.save_freq = save_freq
    self.val_freq = val_freq
    self.checkpoint_path = checkpoint_path
    self.model_name = model_name
    self.current_epoch = 0
    self.epoch_add = epoch_add

  def on_epoch_begin(self, epoch, logs=None):
    self.current_epoch = epoch + self.epoch_add
    # keys = list(logs.keys())
    # print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

  def on_epoch_end(self, epoch, logs=None):
    self.saveTheModel(-1, logs)

  def on_train_batch_end(self, batch, logs=None):
    self.saveTheModel(batch, logs)

  def saveTheModel(self, batch, logs=None):
    if (0 < batch and 0 == batch % self.save_freq) or (0 > batch):
      logging.info(str(datetime.datetime.utcnow()))
    
      if 0 > batch:
        _save_folder = os.path.join(self.checkpoint_path,
                                    self.model_name,
                                    "cp_daily_valid_{:02d}_end".format(self.current_epoch)
                                    )
      else:
        _save_folder = os.path.join(self.checkpoint_path,
                                    self.model_name,
                                    "cp_daily_valid_{:02d}_{:05d}".format(self.current_epoch, batch)
                                    )
      
      fp = os.path.join(_save_folder, "model.h5")
      model.save(fp)
      logging.info(f"Saved model to '{fp}'")
      
      # Save optimizer config
      # c = copy.deepcopy(self.model.optimizer.get_config())

#       fp = os.path.join(_save_folder, "c.pickle")
#       with file_io.FileIO(fp, mode='wb+') as handle:
#         pickle.dump(self.model.optimizer.get_config(), handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#       logging.info(f"Saved optimizer config to '{fp}'")

#       # Save optimizer weights
#       # w = copy.deepcopy(self.model.optimizer.get_weights())

#       fp = os.path.join(_save_folder, "w.pickle")
#       with open(fp, "wb") as handle:
#         # with file_io.FileIO(fp, mode='wb+') as handle:
#         pickle.dump(self.model.optimizer.get_weights(), handle, protocol=pickle.HIGHEST_PROTOCOL)
      
#       logging.info(f"Saved optimizer weights to '{fp}'")
        
      logging.info(f"Did a gc collect: {gc.collect()}")

    # if 0 < batch and 0 == batch % self.val_freq:
    #   print("-------------------------EVAL-------------------------")
    #   model.evaluate(tfgenTest)
    #   print("\n-------------------------EVAL-------------------------")

In [35]:
# If the training is resumed from a previous checkpoint, these epochs can be added to keep model saving consistent
epoch_add = 0

In [36]:
# Define how often the model shall be saved
CALLBACK_EVERY_N_BATCHES = 2000

In [37]:
# Create the custom callback
cc = CustomCallback(checkpoint_path = CHECKPOINT_PATH,
                    model_name = CHKPNT_NAME,
                    save_freq = CALLBACK_EVERY_N_BATCHES,
                    val_freq = CALLBACK_EVERY_N_BATCHES,
                    epoch_add = epoch_add)

In [38]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [39]:
# Todo: Create more elegant solution
log_dir = "gs://ticks_with_indicators_with_volume/logs/TPU/" + CHKPNT_NAME + "_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")

if not COLAB:
    log_dir = os.path.join("/content/CryptoCrystalBall/Data/logs",log_dir.split("/")[-1])

log_dir

'/content/CryptoCrystalBall/Data/logs/FPNWithAttentionBiggerTimebased_3_FutureOnly_60days_GPU_128LB_20230415054519'

In [40]:
# Create a TensorBoard callback for moniting the training progress
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    update_freq=CALLBACK_EVERY_N_BATCHES,
    profile_batch=(5,100)
    )

2023-04-15 05:45:19.094482: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-15 05:45:19.094503: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-15 05:45:19.095203: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2023-04-15 05:45:19.095399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2023-04-15 05:45:19.197628: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-04-15 05:45:19.197822: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1798] CUPTI activity buffer flushed


In [41]:
# Load previous weights if necessary
MODEL_PATH = "/content/CryptoCrystalBall/Notebooks/etf_pastfuture_based/FPNWithAttentionBiggerTimebased_2_FutureOnly_60days_GPU_128LB/cp_daily_valid_05_end/model.h5"

model.load_weights(MODEL_PATH)

In [42]:
Todo important: CustomCallback has to set MODEL_PATH (i think for test and eval saving)

SyntaxError: invalid syntax (3336027124.py, line 1)

In [43]:
# Activate the TensorBoard
%tensorboard --logdir $log_dir --host 0.0.0.0 --port 6006

In [44]:
stop

NameError: name 'stop' is not defined

# Start training

In [45]:
# Train for one epoch on a higher learning rate to get the model going
model.fit(tfGenTraining,
          epochs=1,
          verbose = 1,
          callbacks=[tensorboard_callback, cc])

INFO:root:File 'IE0005042456.csv' loaded, 910 left
INFO:root:File 'IE00BGL86Z12.csv' loaded, 909 left
INFO:root:File 'LU1900067601.csv' loaded, 908 left
INFO:root:File 'IE00B5NLX835.csv' loaded, 907 left
INFO:root:File 'IE0032077012.csv' loaded, 906 left
INFO:root:File 'DE000ETFL557.csv' loaded, 905 left
INFO:root:File 'LU1169821292.csv' loaded, 904 left
INFO:root:File 'DE000A0D8Q23.csv' loaded, 903 left
INFO:root:File 'IE00BYXVGX24.csv' loaded, 902 left
INFO:root:File 'IE00B0M63391.csv' loaded, 901 left
INFO:root:File 'IE00BJ5JP105.csv' loaded, 900 left
2023-04-15 05:46:06.590962: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8700
INFO:root:File 'IE00B4KBBD01.csv' loaded, 899 left
2023-04-15 05:46:07.803527: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
INFO:root:File 'IE00B9M6SJ31.csv' loaded, 898 left


      1/Unknown - 27s 27s/step - loss: 1.2428e-05 - mae: 0.0025

INFO:root:File 'IE00BHZRQY00.csv' loaded, 897 left


      4/Unknown - 28s 285ms/step - loss: 3.0951e-05 - mae: 0.0040

2023-04-15 05:46:16.546428: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-15 05:46:16.546453: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


...

INFO:root:File 'LU1215452928.csv' loaded, 0 left


   4582/Unknown - 1729s 372ms/step - loss: 0.0160 - mae: 0.0046

   4614/Unknown - 1736s 370ms/step - loss: 0.0159 - mae: 0.0046

INFO:root:2023-04-15 06:14:46.278035
INFO:root:Saved model to '/content/bigdata/chk/FPNWithAttentionBiggerTimebased_3_FutureOnly_60days_GPU_128LB/cp_daily_valid_00_end/model.h5'
INFO:root:Did a gc collect: 0


4614/4614 [==============================] - 1738s 371ms/step - loss: 0.0159 - mae: 0.0046


In [46]:
# Decrease the learning rate to avoid overshooting
model.optimizer.learning_rate = 5e-6
logging.info(f"Decreased learning rate: {model.optimizer.learning_rate.numpy()}")

INFO:root:Decreased learning rate: 4.999999873689376e-06


In [47]:
# Create a new CustomCallback with 1 epoch added to the logging file name
epoch_add = 1

cc = CustomCallback(checkpoint_path = CHECKPOINT_PATH,
                    model_name = CHKPNT_NAME,
                    save_freq = CALLBACK_EVERY_N_BATCHES,
                    val_freq = CALLBACK_EVERY_N_BATCHES,
                    epoch_add = epoch_add)

In [48]:
# Train further on with the reduced learning rate
model.fit(tfGenTraining,
          epochs=10,
          verbose = 1,
          callbacks=[tensorboard_callback, cc])

Epoch 1/10


INFO:root:File 'IE0005042456.csv' loaded, 910 left
INFO:root:File 'IE00BGL86Z12.csv' loaded, 909 left
INFO:root:File 'LU1900067601.csv' loaded, 908 left
INFO:root:File 'IE00B5NLX835.csv' loaded, 907 left
INFO:root:File 'IE0032077012.csv' loaded, 906 left
INFO:root:File 'DE000ETFL557.csv' loaded, 905 left
INFO:root:File 'LU1169821292.csv' loaded, 904 left
INFO:root:File 'DE000A0D8Q23.csv' loaded, 903 left
INFO:root:File 'IE00BYXVGX24.csv' loaded, 902 left
INFO:root:File 'IE00B0M63391.csv' loaded, 901 left


      4/Unknown - 14s 203ms/step - loss: 2.0917e-05 - mae: 0.0027

2023-04-15 06:15:01.138804: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-15 06:15:01.138837: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
INFO:root:File 'IE00BJ5JP105.csv' loaded, 900 left


      6/Unknown - 15s 279ms/step - loss: 2.2962e-05 - mae: 0.0029WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1009s vs `on_train_batch_end` time: 0.1596s). Check your callbacks.


...

INFO:root:File 'LU1215452928.csv' loaded, 0 left


4582/4614 [============================>.] - ETA: 11s - loss: 0.0160 - mae: 0.0044

4614/4614 [==============================] - ETA: 0s - loss: 0.0159 - mae: 0.0044

INFO:root:2023-04-15 10:59:51.659342
INFO:root:Saved model to '/content/bigdata/chk/FPNWithAttentionBiggerTimebased_3_FutureOnly_60days_GPU_128LB/cp_daily_valid_10_end/model.h5'
INFO:root:Did a gc collect: 0


4614/4614 [==============================] - 1707s 367ms/step - loss: 0.0159 - mae: 0.0044


In [49]:
stop

NameError: name 'stop' is not defined

# Use the model to predict the future

In [ ]:
# Create a generator for prediction
# Todo: Use same method as for training to avoid different settings

def pythonGeneratorPrediction(fileList):
  # Initialize the FileListToDataStream generator
  dataStream= DataStreamCreator.FileListToDataStream(fileList = fileList,
                                                     batch_size = BATCH_SIZE,
                                                     X_Block_lenght = X_BLOCK_LENGHT,
                                                     y_type_dict=Y_TYPE_DICT,
                                                     shuffle=False,
                                                     parallel_generators = 1,
                                                     random_seed = RANDOM_SEED,
                                                     **DATA_STREAM_PARAMETERS
                                                    )
  
  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStream)
      _X = ne['X']
      _y = _y = ne['y']
    
      # Limit the y signal to avoid excess
      _y /= 10.0
      _y = np.clip(_y, -10.0, 10.0)

      yield (_X, _y)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [ ]:
# Load a file to visualize the prediction results
FILE_FOR_VISUALIZATION = TEST_FILES[4]
FILE_FOR_VISUALIZATION

In [ ]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
tfGenPrediction = tf.data.Dataset.from_generator(lambda: pythonGeneratorPrediction(fileList=[FILE_FOR_VISUALIZATION]), 
                                               output_types = (tf.float32, tf.float32),
                                               output_shapes=(
                                                   (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES),
                                                   (BATCH_SIZE, 4)
                                                   )
                                               )
tfGenPrediction

In [ ]:
# Predict something
p = model.predict(tfGenPrediction)
p *= 10.0
p.shape

In [ ]:
# Store the prediction results in nicely named variables
p_future = p[:,0]
p_future_derivation = p[:,1]

In [ ]:
# Helper method to get the gain data from the past using the generator
def getPastDataFromGen(genIn, predictedShapeIn):
    it = genIn.as_numpy_iterator()

    y_past_data = np.empty((predictedShapeIn[0], 2))
    i = 0

    try:
        while (ne := next(it)):
            y_past_data[i:i+ne[1].shape[0],0] = ne[1][:,0]
            y_past_data[i:i+ne[1].shape[0],1] = ne[1][:,2]

            i += ne[1].shape[0]

    except StopIteration:
        pass

    return y_past_data
    
y_past_data = getPastDataFromGen(tfGenPrediction, p.shape)
y_past_data.shape

In [ ]:
# Todo: Improve plotting of the results

In [ ]:
plt.plot(y_past_data[:,0])

In [ ]:
plt.plot(y_past_data[:,1])

In [ ]:
plt.plot(p_future)

In [ ]:
plt.plot(p_future_derivation)

# Add the predictions to the tick table to visualize them

In [ ]:
# Load the table
tickDF = pd.read_csv(FILE_FOR_VISUALIZATION)
tickDF

In [ ]:
# Because the data is cropped in the batching process (Todo: fix this) and can also not be used fully due to the X-Block and y-data creation,
# the loaded DF has a bigger shape
tickDF.shape[0] - p.shape[0]

In [ ]:
# Crop the DF to meet the prediction
tickDFcropped = tickDF.iloc[-p.shape[0]:, :]
tickDFcropped

In [ ]:
# Plot the prediction again, now over the open price
chart_name="Predicted direction of price movement with open price"
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.set_title(chart_name, fontsize=14)

_ = ax1.plot(p_future, color="green", label="Predicted p_future")
_ = ax1.plot(p_future_derivation, color="red", label="Predicted p_future_derivation")
# _ = ax1.plot(p_neutral, color="gray", label="Predicted neutral")

ax2 = ax1.twinx()

ax2.plot(tickDFcropped.loc[:,'open'].values, color="black", label="Price")

ax1.set_xlabel("Timesteps")
ax1.set_ylabel("Direction")
ax2.set_ylabel("Price")

# Show a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc="upper left")

# Save the figure
# fig_name = "FileListToDataStreamExample_" + chart_name.replace(" ", "_").replace("'", "").replace("(", "").replace(")", "") + ".svg"
# plt.savefig(os.path.join(IMG_SAVE_PATH, fig_name), bbox_inches="tight")

In [ ]:
#@title Buy and sell signal thresholds
BUY_SIGNAL_THRESHOLD_FUTURE = 0.15 #@param {type:"number"}
BUY_SIGNAL_THRESHOLD_PAST = 0.01 #@param {type:"number"}

SELL_SIGNAL_THRESHOLD_PAST = 0.1 #@param {type:"number"}

In [ ]:
#@title Plot buy and sell signals
chart_name="Buy and sell signals"
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.set_title(chart_name)

x_datetimes = [datetime.datetime.fromtimestamp(ts) for ts in tickDFcropped.loc[:,'open'].index]

_ = ax1.plot(x_datetimes, tickDFcropped.loc[:,'open'].values, color="black", label="Price", linewidth=2)

# Color area axis
ax2 = ax1.twinx()

# Convert the float value prediction into rising and falling categories
yCategoryFall = p_future_gain <= SELL_SIGNAL_THRESHOLD_PAST
yCategoryRise = (p_future_gain > p_past_gain) & (p_past_gain <= BUY_SIGNAL_THRESHOLD_FUTURE) #(p_future_gain >= BUY_SIGNAL_THRESHOLD_FUTURE)

ax2.fill_between(x_datetimes, yCategoryFall , alpha=0.5, color="red", label="'Buy' signal")
ax2.fill_between(x_datetimes, yCategoryRise , alpha=0.5, color="green", label="'Sell' signal")

# Remove tick marks and set limits for color area axis
ax2.set_yticks([])
ax2.set_ylim(-0.05,1.05)

ax3 = ax1.twinx()
_ = ax3.plot(x_datetimes, p_future_gain, color="blue", label="Predicted future gain")
# _ = ax3.plot(x_datetimes, 10.0*p_dir_derivation, color="red", label="10.0 * Predicted price direction derivation")
# ax3.set_ylim(-1.05,1.05)

ax3.plot([np.min(x_datetimes), np.max(x_datetimes)], [0.0,0.0], color="gray")

# Show a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax2.legend(lines + lines2 + lines3, labels + labels2 + labels3, loc="upper left")

# Save the figure
# fig_name = chart_name.replace(" ", "_").replace("'", "") + ".svg"
# plt.savefig(os.path.join(IMG_SAVE_PATH, fig_name), bbox_inches="tight")

# Add predictions to tick tables
# Predict the test and eval set
The future information for these sets is predicted using the trained model, additionally the past gain info is acquired using the `FileListToDataStream` generator. It is then stored together with the tick information as .csv files, which then can be used to develop a trading strategy.

```
Todo: Add link to finder notebooks
```

In [ ]:
stop

In [ ]:
# PREDICTED_SAVE_PATH = "/content/dataset/etffancy/PastFutureGainV2/test"

In [ ]:
# Get the model name and the checkpoint name
model_name, checkpoint_name = tuple(MODEL_PATH.split(os.sep)[-3:-1])
print(f"model_name: '{model_name}'")
print(f"checkpoint_name: '{checkpoint_name}'")

In [ ]:
# Replace the latest element in the DATASET_PATH with "predicted"
prediction_save_path = DATASET_PATH.split(os.sep)[:-1]
prediction_save_path += ["predicted"]

# Preserve the root symbol
if prediction_save_path[0] == "":
    prediction_save_path[0] = os.sep

# Add the model info to the path
prediction_save_path += [model_name, checkpoint_name]
    
prediction_save_path

In [ ]:
# Check each path element and create it if necessary
for i in range(1, len(prediction_save_path)):
    partial_path_string = os.path.join(*prediction_save_path[:i+1])

    if not os.path.exists(partial_path_string):
        os.mkdir(partial_path_string)
        
# Create the test and eval save folders
PREDICTION_SAVE_PATH_TEST = os.path.join(*(prediction_save_path + ["test"]))
if not os.path.exists(PREDICTION_SAVE_PATH_TEST):
    os.mkdir(PREDICTION_SAVE_PATH_TEST)

PREDICTION_SAVE_PATH_EVAL = os.path.join(*(prediction_save_path + ["eval"]))
if not os.path.exists(PREDICTION_SAVE_PATH_EVAL):
    os.mkdir(PREDICTION_SAVE_PATH_EVAL)

print(f"PREDICTION_SAVE_PATH_TEST: '{PREDICTION_SAVE_PATH_TEST}'")
print(f"PREDICTION_SAVE_PATH_EVAL: '{PREDICTION_SAVE_PATH_EVAL}'")

In [ ]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
# https://stackoverflow.com/questions/52443273/how-do-you-send-arguments-to-a-generator-function-using-tf-data-dataset-from-gen

fileList = TEST_FILES
PREDICTED_SAVE_PATH = PREDICTION_SAVE_PATH_TEST

pd.options.mode.chained_assignment = None  # default='warn'

for file_index in tqdm(range(0, len(fileList))):
    try:
        filename = fileList[file_index]

        logging.info(f"\nFile name: '{filename}' for index {file_index}")

        tfGenPrediction = tf.data.Dataset.from_generator(lambda: pythonGeneratorPrediction(fileList=[filename]), 
                                                       output_types = (tf.float32, tf.float32),
                                                       output_shapes=(
                                                           (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES),
                                                           (BATCH_SIZE, 4)
                                                           )
                                                       )
        p = model.predict(tfGenPrediction, verbose = 0)

        logging.info(f"p.shape: {p.shape}")

        p_future_gain = p[:,0]
        p_future_gain_derivation = p[:,1]
        
        # Load the tick table
        tickDF = pd.read_csv(filename)
        tickDFcropped = tickDF.iloc[-p.shape[0]:, :]
        
        # Add the predicted future gain
        tickDFcropped.loc[:,"p_future_gain"] = p_future_gain
        tickDFcropped.loc[:,"p_future_gain_derivation"] = p_future_gain_derivation
        
        # Get the (known) past data
        y_past_data = getPastDataFromGen(tfGenPrediction, p.shape)

        # Add the past gain
        tickDFcropped.loc[:,"p_past_gain"] = y_past_data[:,0]
        tickDFcropped.loc[:,"p_past_gain_derivation"] = y_past_data[:,1]

        # Save the DF
        tickDFcropped.to_csv(os.path.join(PREDICTED_SAVE_PATH, filename.split(os.path.sep)[-1]))

        logging.info("Saved the predicted data.")
    except KeyboardInterrupt:
        raise
    except Exception as ex:
        logging.warning(str(ex))

In [ ]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
# https://stackoverflow.com/questions/52443273/how-do-you-send-arguments-to-a-generator-function-using-tf-data-dataset-from-gen

fileList = EVAL_FILES
PREDICTED_SAVE_PATH = PREDICTION_SAVE_PATH_EVAL

# fileList = TEST_FILES
# PREDICTED_SAVE_PATH = "/content/dataset/etffancy/FPNWithAttentionBiggerTimebased_1_FutureOnly/test"

pd.options.mode.chained_assignment = None  # default='warn'

for file_index in tqdm(range(0, len(fileList))):
    try:
        filename = fileList[file_index]

        logging.info(f"\nFile name: '{filename}' for index {file_index}")

        tfGenPrediction = tf.data.Dataset.from_generator(lambda: pythonGeneratorPrediction(fileList=[filename]), 
                                                       output_types = (tf.float32, tf.float32),
                                                       output_shapes=(
                                                           (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES),
                                                           (BATCH_SIZE, 4)
                                                           )
                                                       )
        p = model.predict(tfGenPrediction, verbose = 0)

        logging.info(f"p.shape: {p.shape}")

        p_future_gain = p[:,0]
        p_future_gain_derivation = p[:,1]
        
        # Load the tick table
        tickDF = pd.read_csv(filename)
        tickDFcropped = tickDF.iloc[-p.shape[0]:, :]
        
        # Add the predicted future gain
        tickDFcropped.loc[:,"p_future_gain"] = p_future_gain
        tickDFcropped.loc[:,"p_future_gain_derivation"] = p_future_gain_derivation
        
        # Get the (known) past data
        y_past_data = getPastDataFromGen(tfGenPrediction, p.shape)

        # Add the past gain
        tickDFcropped.loc[:,"p_past_gain"] = y_past_data[:,0]
        tickDFcropped.loc[:,"p_past_gain_derivation"] = y_past_data[:,1]

        # Save the DF
        tickDFcropped.to_csv(os.path.join(PREDICTED_SAVE_PATH, filename.split(os.path.sep)[-1]))

        logging.info("Saved the predicted data.")
    except KeyboardInterrupt:
        raise
    except Exception as ex:
        logging.warning(str(ex))